In [ ]:
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd

# 60-min models: DengAI

# Wrangle Data

## Import

In [ ]:
X = pd.read_csv('data/dengue_features_train.csv', 
                 parse_dates=['week_start_date'],
                 index_col='week_start_date')

## EDA

### Feature Matrix

In [ ]:
X.shape

In [ ]:
X.info()

In [ ]:
X.head()

In [ ]:
X['city'].value_counts()

In [ ]:
X.describe(exclude='number')

### Target

In [ ]:
y = pd.read_csv('data/dengue_labels_train.csv')

In [ ]:
y.head()

In [ ]:
y.describe(exclude='number')

In [ ]:
cols = ['city', 'year', 'weekofyear']
X_values = X[cols].values
y_values = y[cols].values

In [ ]:
X_values[:4]

In [ ]:
y_values[:4]

In [ ]:
assert np.array_equal(X_values, y_values)

In [ ]:
y = pd.read_csv('data/dengue_labels_train.csv', usecols=['total_cases']).set_index(X.index)
y = y['total_cases']

In [ ]:
y.head()

In [ ]:
print(X.shape)

In [ ]:
print(y.shape)

# Split Data

In [ ]:
len(X)*.8

In [ ]:
X.iloc[1164,:]

In [ ]:
cutoff = '2004-11-18'
mask = X.index < cutoff
X_train, y_train = X.loc[mask], y.loc[mask]
X_val, y_val = X.loc[~mask], y.loc[~mask]

In [ ]:
print(X_train.shape)

In [ ]:
print(X_val.shape)

# Establish Baseline

In [ ]:
mean_absolute_error(y_train, [y_train.mean()] * len(y_train))

# Build Model

In [ ]:
model = make_pipeline(
    OneHotEncoder(cols=['city']),
    SimpleImputer(),
    LinearRegression()
)

In [ ]:
model.fit(X_train, y_train);

# Check Metrics

In [ ]:
print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model.predict(X_val)))

# Make our predictions

In [ ]:
X_test = pd.read_csv('data/dengue_features_test.csv', 
                 parse_dates=['week_start_date'],
                 index_col='week_start_date')

In [ ]:
X_test.head()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[:10]